In [2]:
using Symbolics, TermInterface

In [3]:
X = [1 2 3; 4 5 6; 7 8 9]

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [4]:
X[2,2]

5

In [5]:
X[1:2,1:3]

2×3 Matrix{Int64}:
 1  2  3
 4  5  6

In [6]:
X[map(x -> x:3, [1, 2])...]

3×2 Matrix{Int64}:
 2  3
 5  6
 8  9

In [7]:
X[(x:3 for x in [2, 1])...]

2×3 Matrix{Int64}:
 4  5  6
 7  8  9

`@edit @variables`

In [8]:
function HM(sz::Int...; symbol::Symbol)
    (@variables $symbol[map(x -> 1:x, sz)...])[1]
end

function HM(sz::AbstractRange{Int}...; symbol::Symbol)
    (@variables $symbol[sz...])[1]
end

HM (generic function with 2 methods)

In [9]:
A = HM(2, 3, symbol=:a)

a[1:2,1:3]

In [10]:
A[1,2]*A[2,1]

a[1, 2]*a[2, 1]

In [11]:
typeof(A)

Symbolics.Arr{Num, 2}

In [12]:
eltypes(::Type{Symbolics.Arr{T,D}}) where {T,D} = (T, D)
eltypes(x::Symbolics.Arr) = eltypes(typeof(x))
eltypes(A)

(Num, 2)

In [13]:
A |> Symbolics.unwrap |> nameof

:a

In [14]:
A |> Symbolics.unwrap |> TermInterface.metadata

Base.ImmutableDict{DataType, Any} with 2 entries:
  VariableSource => (:variables, :a)
  ArrayShapeCtx  => (1:2, 1:3)

In [15]:
axesofA = axes(A)

(1:2, 1:3)

In [16]:
B = HM(2:3, 0:2:4, 1:4, symbol=:bb)

bb[2:3,0:2:4,1:4]

In [17]:
B[1,1,1]*B[2,3,2]

bb[2, 0, 1]*bb[3, 4, 2]

In [18]:
typeof(B), eltypes(B)

(Symbolics.Arr{Num, 3}, (Num, 3))

In [19]:
axes(B) .|> [(x->x) length]

3×2 Matrix{Any}:
 2:3    2
 0:2:4  3
 1:4    4

In [20]:
[firstindex.([B],[1 2 3]) ; lastindex.([B],[1 2 3])]

2×3 Matrix{Int64}:
 2  0  1
 3  4  4

In [21]:
# permutedims(ans, (2,1))
ans'

3×2 adjoint(::Matrix{Int64}) with eltype Int64:
 2  3
 0  4
 1  4

In [36]:
Base.similar(A::AbstractArray, T::Type, dims::Tuple{AbstractRange{I}, Vararg{AbstractRange{I}}}) where {I<:Int} = similar(A, T, tuple(map(x -> length(x), dims)...))
    
Base.similar(f::Union{Function,DataType}, dims::Tuple{AbstractRange{I}, Vararg{AbstractRange{I}}}) where {I<:Int} = similar(f, tuple(map(x -> length(x), dims)...))

function printHM(Hx::Symbolics.Arr{T,2}) where {T}
    println("2 method called! ", Hx)
    if any(firstindex.([Hx],[1:2...]') .!= 1)
        r,c = length.(axes(Hx))
        [Hx[i+r*j] for i in 1:r, j in 0:(c-1)]
    else
        [x for x in Hx]
    end
end

function printHM(Hx::Symbolics.Arr{T,D}, dim1=0, dim2=1, dpths=nothing) where {T,D}
    println("general method called! ", Hx)
    if any(firstindex.([Hx],[1:D...]') .!= 1)
        [Hx[i] for i in CartesianIndex(repeat([1],D)...):CartesianIndex(length.(axes(Hx))...)]
    else
        [x for x in Hx]
    end
end


#     try
#         [x for x in Hx]
#     catch e
#         if isa(e, BoundsError)
#             println("Implementation to follow soon")
#             [0]
#         else
#             throw(e)
#         end
#     end

printHM (generic function with 5 methods)

In [23]:
printHM(A)

2 method called! a[1:2,1:3]


2×3 Matrix{Num}:
 a[1, 1]  a[1, 2]  a[1, 3]
 a[2, 1]  a[2, 2]  a[2, 3]

In [24]:
printHM(B)

general method called! bb[2:3,0:2:4,1:4]


2×3×4 Array{Num, 3}:
[:, :, 1] =
 bb[2, 0, 1]  bb[2, 2, 1]  bb[2, 4, 1]
 bb[3, 0, 1]  bb[3, 2, 1]  bb[3, 4, 1]

[:, :, 2] =
 bb[2, 0, 2]  bb[2, 2, 2]  bb[2, 4, 2]
 bb[3, 0, 2]  bb[3, 2, 2]  bb[3, 4, 2]

[:, :, 3] =
 bb[2, 0, 3]  bb[2, 2, 3]  bb[2, 4, 3]
 bb[3, 0, 3]  bb[3, 2, 3]  bb[3, 4, 3]

[:, :, 4] =
 bb[2, 0, 4]  bb[2, 2, 4]  bb[2, 4, 4]
 bb[3, 0, 4]  bb[3, 2, 4]  bb[3, 4, 4]

In [25]:
C = HM(3, 2, 4, symbol=:c)

c[1:3,1:2,1:4]

In [26]:
printHM(C)

general method called! c[1:3,1:2,1:4]


3×2×4 Array{Num, 3}:
[:, :, 1] =
 c[1, 1, 1]  c[1, 2, 1]
 c[2, 1, 1]  c[2, 2, 1]
 c[3, 1, 1]  c[3, 2, 1]

[:, :, 2] =
 c[1, 1, 2]  c[1, 2, 2]
 c[2, 1, 2]  c[2, 2, 2]
 c[3, 1, 2]  c[3, 2, 2]

[:, :, 3] =
 c[1, 1, 3]  c[1, 2, 3]
 c[2, 1, 3]  c[2, 2, 3]
 c[3, 1, 3]  c[3, 2, 3]

[:, :, 4] =
 c[1, 1, 4]  c[1, 2, 4]
 c[2, 1, 4]  c[2, 2, 4]
 c[3, 1, 4]  c[3, 2, 4]

In [44]:
D = HM(2:4, 1:2, symbol=:d)

d[2:4,1:2]

In [45]:
printHM(D)

2 method called! d[2:4,1:2]


3×2 Matrix{Num}:
 d[2, 1]  d[2, 2]
 d[3, 1]  d[3, 2]
 d[4, 1]  d[4, 2]